In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install openimages
clear_output()

In [ ]:
!mkdir data

In [ ]:
# !oi_download_images --csv_dir data --base_dir data --labels "Vehicle registration plate" --format "darknet" --limit 3000

In [ ]:
from openimages.download import download_dataset
download_dataset("data", ["Vehicle registration plate"], annotation_format="darknet",limit=3000)

2020-12-31  09:02:45 INFO NumExpr defaulting to 2 threads.
2020-12-31  09:02:47 INFO Downloading 3000 train images for class 'vehicle registration plate'
100%|██████████| 3000/3000 [00:28<00:00, 104.74it/s]
2020-12-31  09:03:16 INFO Creating 3000 train annotations (darknet) for class 'vehicle registration plate'
100%|██████████| 3000/3000 [00:01<00:00, 2654.92it/s]


{'vehicle registration plate': {'annotations_dir': 'data/vehicle registration plate/darknet',
  'images_dir': 'data/vehicle registration plate/images'}}

In [ ]:
!mkdir dataset dataset/images dataset/labels
!mkdir dataset/images/train dataset/images/val
!mkdir dataset/labels/train dataset/labels/val

In [ ]:
import os
all_ = [i for i in os.listdir('/content/data/vehicle registration plate/images')]
train = all_[:2400]
test = all_[2400:]
len(all_),len(train),len(test)

(3000, 2400, 600)

In [ ]:
!cp data/vehicle\ registration\ plate/images/* dataset/images/train/
!cp data/vehicle\ registration\ plate/images/* dataset/images/val/
!cp data/vehicle\ registration\ plate/darknet/* dataset/labels/train/
!cp data/vehicle\ registration\ plate/darknet/* dataset/labels/val/

In [ ]:
for i in os.listdir("/content/dataset/images/train"):
  if i not in train:
    os.remove(f"/content/dataset/images/train/{i}")
for i in os.listdir("/content/dataset/images/val"):
  if i not in test:
    os.remove(f"/content/dataset/images/val/{i}")

In [ ]:
len(os.listdir("/content/dataset/images/train")),len(os.listdir("/content/dataset/images/val"))

(2400, 600)

In [ ]:
for i in os.listdir("/content/data/vehicle registration plate/images"):
  if i not in train:
    os.remove(f"/content/dataset/labels/train/{i.split('.')[0]}.txt")
for i in os.listdir("/content/data/vehicle registration plate/images"):
  if i not in test:
    os.remove(f"/content/dataset/labels/val/{i.split('.')[0]}.txt")

In [ ]:
len(os.listdir("/content/dataset/labels/train")),len(os.listdir("/content/dataset/labels/val"))

(2400, 600)

Yolo V5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 3773 (delta 68), reused 69 (delta 33), pack-reused 3662
Receiving objects: 100% (3773/3773), 7.14 MiB | 35.51 MiB/s, done.
Resolving deltas: 100% (2523/2523), done.
/content/yolov5
     |████████████████████████████████| 276kB 11.0MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=44b16bb2ef8679b354748f2f18f41980f6fa13b6341cbc84b453b9ab497ebd1f
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


In [ ]:
!cp /content/yolov5/data/coco128.yaml ../

train: ../coco128/images/train2017/  
val: ../coco128/images/train2017/  

# number of classes
nc: 1

# class names
names: ['lincense_plate']


In [ ]:
!python train.py --img 640 --batch 64 --epochs 30 --data /content/data.yaml \
 --weights yolov5s.pt --nosave --cache

Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079.75MB)

Namespace(adam=False, batch_size=64, bucket='', cache_images=True, cfg='', data='/content/data.yaml', device='', epochs=30, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=True, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/exp3', single_cls=False, sync_bn=False, total_batch_size=64, weights='yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4

Export

In [ ]:
!pip install onnx
!pip install coremltools
clear_output()

In [ ]:
!python models/export.py --weights runs/train/exp3/weights/last.pt --img 640 --batch 1

Namespace(batch_size=1, img_size=[640, 640], weights='runs/train/exp3/weights/last.pt')
Fusing layers... 
Model Summary: 232 layers, 7246518 parameters, 0 gradients, 16.8 GFLOPS

Starting TorchScript export with torch 1.7.0+cu101...
/usr/local/lib/python3.6/dist-packages/torch/jit/_trace.py:940: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  _force_outplace,
TorchScript export success, saved as runs/train/exp3/weights/last.torchscript.pt

Starting ONNX export with onnx 1.8.0...
ONNX export success, saved as runs/train/exp3/weights/last.onnx
scikit-learn version 0.22.2.post1 is not supported. Minimum required v